<h2 align = "center">Sprawozdanie mofit</h2>
<h4 align="center">Łukasz Puchała, Karol Grzywa</h4>

In [6]:
import numpy as np
from numba import njit
import math
import matplotlib.pyplot as plt
from math import sin, cos, pi, e, exp
from scipy.linalg import eigh
import Utilis

m = 0.067 # m0
h = 1
at2meV = 27211.6
ab = 0.05292
j = 0 + 1j
at2s = 2.42e-5*1e-12

L = 100/ab
N = 2
a = L/(2*N)

In [ ]:
grid_N2_L100 = Utilis.ReadFile('Files/wezly_N_2_L_100.dat')

,0,1,2
0,1,-50.0,-50.0
1,2,-25.0,-50.0
2,3,0.0,-50.0
3,4,25.0,-50.0
4,5,-50.0,-25.0
...,...,...,...
76,77,12.5,50.0
77,78,12.5,37.5
78,79,50.0,37.5
79,80,37.5,50.0
